# Job posting gathering

In [101]:
import requests
from bs4 import BeautifulSoup
import pprint
import time
import itertools

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException


In [105]:
def press_button_until_gone(button_selector, url=None, wait_time=2, 
                            pre_click_wait=0, post_click_wait=0,
                            scroll=True, driver = None, close_driver=True, open_page = True):
    """
    Continuously presses a button on a webpage until it no longer exists.
    """
    if driver is None:
        driver = webdriver.Firefox()
    if url and open_page:
        driver.get(url)
    step_counter = 0
    
    try:
        time.sleep(pre_click_wait)
        while step_counter < 100:
            try:
                if scroll:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(wait_time/2)
                button = driver.find_element("css selector", button_selector)
                button.click()
                step_counter += 1
                time.sleep(wait_time/2)
            
            except (NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException):
                break

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
    finally:
        if close_driver:
            driver.quit()
    
    return soup


def close_cookies(button_selector, url=None, click_wait=15, post_click_wait = 0,
                  pre_scroll=False, post_scroll=False, 
                  driver = None, close_driver=True, open_page=True):
    """
    Continuously presses a button on a webpage until it no longer exists.
    """
    if driver is None:
        driver = webdriver.Firefox()
    if url and open_page:
        driver.get(url)
    step_counter = 0
    
    try:
        WebDriverWait(driver, click_wait).until(
                expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, button_selector))
        )
        try:
            if pre_scroll:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            button = driver.find_element("css selector", button_selector)
            button.click()
            if post_scroll:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            step_counter += 1
        
        except (NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException):
            pass
            
        if post_click_wait:
            time.sleep(post_click_wait)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
    finally:
        if close_driver:
            driver.quit()
    return soup

In [99]:
driver = webdriver.Firefox()
driver.get("https://www.karriere.at/jobs/data-mining/wien-und-umgebung")

In [103]:
url = "https://www.karriere.at/jobs/data-mining/wien-und-umgebung"  # Replace with your target webpage
button_selector = ".onetrust-close-btn-handler"  # Replace with the actual CSS selector of the button
final_page_soup = close_cookies(url, button_selector, click_wait=15, driver=driver, close_driver=False)

In [104]:
final_page_soup

<html data-event-category="page: jobs" lang="de"><head> <script async="" src="https://www.googletagmanager.com/gtag/destination?id=DC-9907740&amp;l=dataLayer&amp;cx=c&amp;gtm=45He4cc1v812161417za200" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/destination?id=DC-9649457&amp;l=dataLayer&amp;cx=c&amp;gtm=45He4cc1v812161417za200" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-VLJ9DMQTZG&amp;l=dataLayer&amp;cx=c&amp;gtm=45He4cc1v812161417za200" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-5WGVNH5"></script><script>if('performance' in this && 'mark' in this.performance) { performance.mark("markup-preconnect");
}</script> <link href="//kcdn.at" rel="preconnect"/> <link href="//www.google-analytics.com" rel="preconnect"/> <script> if (typeof window.dataLayer !== 'object') { window.dataLayer = window.dataLayer || []; } (function(w,d,s,l,i){

In [94]:
url = "https://www.karriere.at/jobs/data-mining/wien-und-umgebung"  # Replace with your target webpage
button_selector = ".m-loadMoreJobsButton__button"  # Replace with the actual CSS selector of the button
final_page_soup = press_button_until_gone(url, button_selector, wait_time=4, first_click_wait=10, driver=driver, close_driver=False)

Button clicked 1 times.
Button no longer exists or is not clickable.


scroll logic function?

In [106]:
def scroll_scrape_websites(urls, wait_time = 2):
    driver = webdriver.Firefox() #only closed after all websites are scraped

    all_soups = []
    for url in urls:
        driver.get(url)
        last_height = driver.execute_script("return document.body.scrollHeight")
        step_counter = 0

        while step_counter<100:
            step_counter += 1
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  #Scroll to "height"-> bottom of the page
            time.sleep(wait_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                if step_counter == 1:
                    continue
                else:
                    break
            last_height = new_height

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        all_soups.append(soup)
    driver.quit()
    return all_soups


def scroll_scrape_website(url=None, driver = None, close_driver = True, wait_time = 2, open_page=True):
    if driver is None:
        driver = webdriver.Firefox()
    if url and open_page:
        driver.get(url)
        
    last_height = driver.execute_script("return document.body.scrollHeight")
    step_counter = 0

    while step_counter<100:
        step_counter += 1
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  #Scroll to "height"-> bottom of the page
        time.sleep(wait_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  #We didn't scroll anymore
            if step_counter == 1: #We retry if this is first iteration. This is because the website may not have loaded properly
                continue
            else:
                break
        last_height = new_height
        
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    if close_driver:
        driver.quit()
    return soup

def requests_responses(urls, return_kind = 'soups', https = False, headers = None):
    responses = []
    session = requests.Session()
    if https:
        if 'User-Agent' not in headers:
            headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    for url in urls:
        response = session.get(url, headers = headers)
        responses.append(response)
        
    if return_kind == 'responses':
        return responses
    if return_kind == 'soups':
        soups = [BeautifulSoup(response.content, 'html.parser') for response in responses]
        return soups
    
    raise ValueError(f'Unknown return_kind: {return_kind}')

## Website: karriere.at

In [49]:
def process_webpage(urls, usecase='http', **kwargs):
    if usecase == 'http':
        soups = requests_responses(urls)
        return soups
    
    if usecase == 'https':
        soups = requests_responses(urls, https = True)
        return soups
    
    if usecase == 'selenium':
        soups = scroll_scrape_websites(urls, **kwargs)
        return soups
    
    if usecase == 'http_selenium':
        """Use: e.g. when we do not always want to use Selenium, we first try HTTP(S) and if there is more to load with JS, we use Selenium"""
        #planned use: first try to scrape with http. If a defined condition function returns true, try with selenium
        pass
    
    raise ValueError(f'Unknown method: {usecase}')

def url_builder(base_url, slash_list=[]):
    """Expected strings in slash list (potentially will be extended to dicts)"""
    url = base_url
    if url[-1] == '/':
        url = url[:-1]
    for slash in slash_list:
        url += '/' + slash
    return url

def urls_builder(base_url, slash_elements_list = [], zipped = True, all_combinations = False):
    """expecting list of lists"""
    urls = []
    if zipped:
        if all_combinations:
            slash_lists = list(set(itertools.product(*slash_elements_list)))
        else:
            slash_lists = list(zip(*slash_elements_list))
    else:
        
        slash_lists = slash_elements_list
    for slash_list in slash_lists:
            urls.append(url_builder(base_url, slash_list))
    return urls

def gather_data_karriereat():
    titlewords =["machine-learning","machine-learning-engineer","machine-learning-scientist",
     "ML-scientist", "ML-engineer", "ML-researcher", "ML-developer", "ML-AI",
     "AI-engineer", "AI-scientist", "AI-researcher", "AI-developer", "AI-ML", 
     "data-science","data-scientist", "data-mining"
     "data-engineer", "data-engineering", "data-engineering-developer",
     "data-analysis", "data-analytics", "data-analyst",
     "business-intelligence", "business-intelligence-analyst", "bi-analyst", "business-analyst",
     ]
    locations =["wien-und-umgebung"]*len(titlewords)
    urls_links = urls_builder('https://www.karriere.at/jobs', [titlewords, locations], zipped = True, all_combinations = False)
    pattern = 'div.m-jobsListItem__container div.m-jobsListItem__dataContainer h2.m-jobsListItem__title a.m-jobsListItem__titleLink'

In [52]:
titlewords =["machine-learning","machine-learning-engineer","machine-learning-scientist",
     "ML-scientist", "ML-engineer", "ML-researcher", "ML-developer", "ML-AI",
     "AI-engineer", "AI-scientist", "AI-researcher", "AI-developer", "AI-ML", 
     "data-science","data-scientist",
     "data-engineer", "data-engineering", "data-engineering-developer",
     "data-analysis", "data-analytics", "data-analyst",
     "business-intelligence", "business-intelligence-analyst", "bi-analyst", "business-analyst",
     ]
locations =["wien-und-umgebung"]*len(titlewords)
urls_jobs = urls_builder('https://www.karriere.at/jobs', [titlewords, locations], zipped = True, all_combinations = False)


In [53]:
urls_jobs

['https://www.karriere.at/jobs/machine-learning/wien-und-umgebung',
 'https://www.karriere.at/jobs/machine-learning-engineer/wien-und-umgebung',
 'https://www.karriere.at/jobs/machine-learning-scientist/wien-und-umgebung',
 'https://www.karriere.at/jobs/ML-scientist/wien-und-umgebung',
 'https://www.karriere.at/jobs/ML-engineer/wien-und-umgebung',
 'https://www.karriere.at/jobs/ML-researcher/wien-und-umgebung',
 'https://www.karriere.at/jobs/ML-developer/wien-und-umgebung',
 'https://www.karriere.at/jobs/ML-AI/wien-und-umgebung',
 'https://www.karriere.at/jobs/AI-engineer/wien-und-umgebung',
 'https://www.karriere.at/jobs/AI-scientist/wien-und-umgebung',
 'https://www.karriere.at/jobs/AI-researcher/wien-und-umgebung',
 'https://www.karriere.at/jobs/AI-developer/wien-und-umgebung',
 'https://www.karriere.at/jobs/AI-ML/wien-und-umgebung',
 'https://www.karriere.at/jobs/data-science/wien-und-umgebung',
 'https://www.karriere.at/jobs/data-scientist/wien-und-umgebung',
 'https://www.karrier

In [2]:
url = 'https://www.karriere.at/jobs/machine-learning/wien-und-umgebung'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [68]:
soup.select('div.m-jobsListItem__container div.m-jobsListItem__dataContainer h2.m-jobsListItem__title a.m-jobsListItem__titleLink')[0].text

' Consultant Technology Strategy & Advisory (all genders) '

Can also select `'.m-jobsList__item'`

In [76]:
pattern = '.m-jobsListItem--isSplitView'
jobs = soup.select(pattern)
pprint.pprint(jobs)

[<div class="m-jobsListItem m-jobsListItem--active m-jobsListItem--isSplitView" data-company-id="13540" data-id="7388680" data-isactive="true" data-page="jobs" data-show-new-job-item="true"> <div class="m-jobsListItem__container m-jobsListItem__container--hasLogo"> <div class="m-jobsListItem__imageContainer"> <span class="m-jobsListItem__imageLink"> <img alt="Universität für Bodenkultur Wien" class="m-jobsListItem__logo" height="61" loading="lazy" src="//kcdn.at/company/13540/411301/logo-universit--t-f-bodenkultur-wien.companysquare.gif" width="61"/> </span> </div> <div class="m-jobsListItem__dataContainer"> <div class="m-jobsListItem__badges"> <span class="m-jobsListItem__smartApplyBadge"> <div class="m-spot"> <svg class="m-spot__svg" viewbox="0 0 96.067 8.005"><g fill="none"><g fill="#8BC72A"><path d="M95.8 3.6L89.8.1a.4.4 0 0 0-.7.3l.2.4 4.9 2.8c.3.2.3.6 0 .8l-4.9 2.8a.5.5 0 0 0-.2.4c0 .3.4.5.7.3l6.1-3.5c.3-.2.3-.6 0-.8z"></path><path d="M92.5 3.6L86.3.1a.4.4 0 0 0-.7.3v7.2c0 .3.4.5

Need to use Selenium to get full page

In [77]:
jobs[0]['data-id'], jobs[0].select('div.m-jobsListItem__container')[0].select('div.m-jobsListItem__dataContainer')[0].select('h2.m-jobsListItem__title')[0].select('a.m-jobsListItem__titleLink')[0]['href']

('7388680', 'https://www.karriere.at/jobs/7388680')

In [70]:
for job_posting in jobs:
    url = job_posting.select('div.m-jobsListItem__container')[0].select('div.m-jobsListItem__dataContainer')[0].select('h2.m-jobsListItem__title')[0].select('a.m-jobsListItem__titleLink')[0]['href']
    print(url)

https://www.karriere.at/jobs/7388680
https://www.karriere.at/jobs/7329805
https://www.karriere.at/jobs/7308601
https://www.karriere.at/jobs/7174687
https://www.karriere.at/jobs/7174528
https://www.karriere.at/jobs/7330669
https://www.karriere.at/jobs/7372609
https://www.karriere.at/jobs/7355608
https://www.karriere.at/jobs/7378243
https://www.karriere.at/jobs/7352026
https://www.karriere.at/jobs/7323589
https://www.karriere.at/jobs/7380166
https://www.karriere.at/jobs/7379068
https://www.karriere.at/jobs/7376827
https://www.karriere.at/jobs/7339042


## TieTalent

In [79]:
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = 'https://tietalent.com/en/jobs?search=positions%5B0%5D%3DData_Engineer_5%26positions%5B1%5D%3DData_Analyst_36%26positions%5B2%5D%3DData_Scientist_37%26positions%5B3%5D%3DMachine_Learning_7%26positions%5B4%5D%3DBusiness_Intelligence_39%26positions%5B5%5D%3DNLP_14%26locations%5B0%5D%3DVienna_Vienna_Austria_304'
response = session.get(url, headers=headers)

Need to try page 2, 3, ... aswell

In [80]:
soup = BeautifulSoup(response.content, 'html.parser')